# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = os.path.join('..', 'output_data', 'cities.csv')

csv_read = pd.read_csv(csv_path)
csv_read.head()


,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,6185377,Yellowknife,62.4560,-114.3525,-1.42,51,75,4.12,CA,2021-04-30 01:54:21
1,5364271,Laguna,38.4210,-121.4238,30.30,16,1,3.09,US,2021-04-30 01:52:45
2,4030556,Rikitea,-23.1203,-134.9692,25.03,63,77,5.74,PF,2021-04-30 01:54:18
3,3369157,Cape Town,-33.9258,18.4232,12.96,93,0,2.57,ZA,2021-04-30 01:50:17
4,933649,Khakhea,-24.6895,23.4940,16.76,82,62,3.66,BW,2021-04-30 02:04:12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
Lat_Long = csv_read[["Lat", "Lng"]]
humidity = csv_read["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(Lat_Long, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# I adjusted the data quite a bit

new_df = csv_read[(csv_read['Max Temp'] <= 50) & (csv_read['Max Temp'] >= 25)] 
new_df = new_df[new_df['Wind Speed'] <= 10]
new_df = new_df[new_df['Cloudiness'] <= 25]

new_df.head()


,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,5364271,Laguna,38.4210,-121.4238,30.30,16,1,3.09,US,2021-04-30 01:52:45
6,2068110,Kununurra,-15.7667,128.7333,32.00,29,0,4.63,AU,2021-04-30 01:59:20
28,1151464,Phang Nga,8.4509,98.5298,28.26,83,20,2.57,TH,2021-04-30 02:04:20
45,2063042,Port Hedland,-20.3167,118.5667,34.00,20,0,2.57,AU,2021-04-30 01:54:41
46,1783873,Zhumadian,32.9794,114.0294,25.21,49,0,6.44,CN,2021-04-30 02:04:25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = new_df

hotel_df['Hotel Name'] = ' '
hotel_df

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,5364271,Laguna,38.4210,-121.4238,30.30,16,1,3.09,US,2021-04-30 01:52:45,
6,2068110,Kununurra,-15.7667,128.7333,32.00,29,0,4.63,AU,2021-04-30 01:59:20,
28,1151464,Phang Nga,8.4509,98.5298,28.26,83,20,2.57,TH,2021-04-30 02:04:20,
45,2063042,Port Hedland,-20.3167,118.5667,34.00,20,0,2.57,AU,2021-04-30 01:54:41,
46,1783873,Zhumadian,32.9794,114.0294,25.21,49,0,6.44,CN,2021-04-30 02:04:25,
55,1258338,Ratnagiri,16.9833,73.3000,28.64,71,16,1.67,IN,2021-04-30 02:04:29,
75,1270961,Gopālpur,19.2667,84.9167,28.63,74,0,6.16,IN,2021-04-30 02:04:36,
76,1735106,George Town,5.4112,100.3354,28.04,83,20,2.06,MY,2021-04-30 01:54:10,
101,1252948,Warangal,18.0000,79.5833,28.84,54,4,5.90,IN,2021-04-30 02:04:45,
138,4020109,Atuona,-9.8000,-139.0333,26.30,69,13,1.98,PF,2021-04-30 02:04:59,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        params = {'keyword': 'hotel', 'radius': 5000, 'key': g_key}
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f'{lat}, {lng}'
        hotel_data = requests.get(url, params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = hotel_data['results'][0]['name']
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = 'No result'
        
hotel_df

,ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,5364271,Laguna,38.4210,-121.4238,30.30,16,1,3.09,US,2021-04-30 01:52:45,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
6,2068110,Kununurra,-15.7667,128.7333,32.00,29,0,4.63,AU,2021-04-30 01:59:20,Freshwater East Kimberley Apartments | Kununur...
28,1151464,Phang Nga,8.4509,98.5298,28.26,83,20,2.57,TH,2021-04-30 02:04:20,Blue Mountain Phangnga Resort and Restaurant
45,2063042,Port Hedland,-20.3167,118.5667,34.00,20,0,2.57,AU,2021-04-30 01:54:41,Hedland Hotel
46,1783873,Zhumadian,32.9794,114.0294,25.21,49,0,6.44,CN,2021-04-30 02:04:25,IKE INTERNATIONAL HOTEL
55,1258338,Ratnagiri,16.9833,73.3000,28.64,71,16,1.67,IN,2021-04-30 02:04:29,"Kohinoor Samudra Beach Resort, Ratnagiri"
75,1270961,Gopālpur,19.2667,84.9167,28.63,74,0,6.16,IN,2021-04-30 02:04:36,"MAYFAIR Palm Beach Resort, Gopalpur"
76,1735106,George Town,5.4112,100.3354,28.04,83,20,2.06,MY,2021-04-30 01:54:10,"Eastern & Oriental Hotel, Penang"
101,1252948,Warangal,18.0000,79.5833,28.84,54,4,5.90,IN,2021-04-30 02:04:45,Sharanya Hotel
138,4020109,Atuona,-9.8000,-139.0333,26.30,69,13,1.98,PF,2021-04-30 02:04:59,Hôtel Hiva Oa Hanakee Lodge


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))